In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re

In [2]:
main_link_hh = 'https://hh.ru'
main_link_sj = 'https://www.superjob.ru'

In [3]:
header = {'Accept': '*/*',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0'
}

В переменную function_name вводить желаемую должность

In [4]:
function_name = 'python'

In [5]:
function_name_correct = function_name#.replace(' ', '+')

In [6]:
parameter = {
    'text': function_name_correct,
    'search_period': "30" #1,3,7,30
}

In [7]:
session = requests.session()

In [8]:
def parse_hh(html):
    data = list()
    vacancys_block = html.find('div', {'class':"vacancy-serp vacancy-serp_xs-mode"})
    pattern = re.compile('vacancy-serp-item.*')
    vacancy_block = vacancys_block.findChildren('div', {'class':pattern}, recursive=False)
    for vacancy in vacancy_block:
        dict_ = {}
        
        dict_['name'] = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'}).text
        dict_['href'] = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
        #dict_['company'] = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'}).text
        dict_['company'] = vacancy.findAll('div', {'class': 'vacancy-serp-item__meta-info'})[0].text.replace('\xa0', ' ')
        dict_['position'] = vacancy.findAll('div', {'class': 'vacancy-serp-item__meta-info'})[1].text
        dict_['site'] = main_link_hh

        finance = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).text.replace('\xa0', ' ')
        if not finance:
            dict_['finance_min'] = 'no data'
            dict_['finance_max'] = 'no data'
            dict_['valuta'] = 'no data'
        else:        
            pattern_number = re.compile('[0-9]{1,3}\s{1}[0-9]{3}')
            pattren_valuta = re.compile('руб\.|USD|EUR|KZT|грн.')
            pattren_ot = re.compile('от')
            pattren_do = re.compile('до')

            number = re.findall(pattern_number, finance)
            valuta = re.findall(pattren_valuta, finance)
            if valuta:
                valuta = valuta[0]
            

            if len(number) == 2:
                dict_['finance_min'] = number[0]
                dict_['finance_max'] = number[1]
                dict_['valuta'] = valuta
            elif len(number) == 1:
                ot = re.findall(pattren_ot, finance)
                do = re.findall(pattren_do, finance)
                if not do:
                    dict_['finance_min'] = number[0]
                    dict_['finance_max'] = '-'
                    dict_['valuta'] = valuta
                elif not ot:
                    dict_['finance_min'] = '-'
                    dict_['finance_max'] = number[0]
                    dict_['valuta'] = valuta
        
        #print(dict_)
        data.append(dict_) 
    return data

In [9]:
def parse_sj(html):
    data = list()
    vacancy_blocks = html.find('div', {'style':"display:block"})
    #print(vacancy_blocks)
    
    vacancy_block = vacancy_blocks.findChildren('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'}, recursive=False)
    #print(vacancy_block)
    
    for vacancy in vacancy_block:
        dict_ = {}
        
        pattern_name = re.compile('icMQ_ _1QIBo .*')
        name_module = vacancy.find('a', {'class': pattern_name})
        dict_['name'] = name_module.text
        dict_['href'] = main_link_sj + name_module['href']
        
        pattern_company = re.compile('icMQ_ _205Zx .*')
        company_module = vacancy.find('a', {'class': pattern_company})
        if company_module:
            dict_['company'] = company_module.text
        else:
            dict_['company'] = None
        
        pattern_position = re.compile('_3mfro f-test-text-company-item-location.*')
        position_module = vacancy.find('span', {'class': pattern_position})
        if company_module:
            dict_['position'] = position_module.text
        else:
            dict_['position'] = None

        
        dict_['site'] = main_link_sj

        
        pattern_finance = re.compile('_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz')
        finance_module = vacancy.find('span', {'class': pattern_finance})
        finance = finance_module.text.replace('\xa0', ' ')
        #dict_['finance'] = finance
        if finance == 'По договорённости':
            dict_['finance_min'] = 'По договорённости'
            dict_['finance_max'] = 'По договорённости'
            dict_['valuta'] = 'no data'
        else:        
            pattern_number = re.compile('[0-9]{1,3}\s{1}[0-9]{3}')
            pattren_valuta = re.compile('₽')
            pattren_ot = re.compile('от')
            pattren_do = re.compile('до')

            number = re.findall(pattern_number, finance)
            valuta = re.findall(pattren_valuta, finance)
            if valuta:
                valuta = valuta[0] 
            if len(number) == 2:
                dict_['finance_min'] = number[0]
                dict_['finance_max'] = number[1]
                dict_['valuta'] = valuta
            elif len(number) == 1:
                ot = re.findall(pattren_ot, finance)
                do = re.findall(pattren_do, finance)
                if not do:
                    dict_['finance_min'] = number[0]
                    dict_['finance_max'] = '-'
                    dict_['valuta'] = valuta
                elif not ot:
                    dict_['finance_min'] = '-'
                    dict_['finance_max'] = number[0]
                    dict_['valuta'] = valuta
        
        #print(dict_)
        data.append(dict_) 
    return data

In [10]:
def get_html_proxies(link, header):
    proxies = {
    "http": ""
}
    response = session.get(link, headers=header, proxies=proxies)
    # print(response.status_code)
    html = bs(response.text,'lxml')
    print(response.status_code)
    return html

In [11]:
def get_html(link, header):
    response = session.get(link, headers=header)
    # print(response.status_code)
    html = bs(response.text,'lxml')
    print(response.status_code)
    return html

In [12]:
def get_next_link_hh(html):
    bufer = html.find('a', {'class':"bloko-button HH-Pager-Controls-Next HH-Pager-Control"})
    if bufer:
        return main_link_hh + bufer['href']
        link = bufer['href']
    else:
        return ''

In [13]:
def get_next_link_sj(html):
    bufer = html.find('a', {'class':"icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe"})
    if bufer:
        return main_link_sj + bufer['href']
        link = bufer['href']
    else:
        return ''

In [14]:
result = list()
link = main_link_hh + f'/search/vacancy?search_period=30&salary=&st=searchVacancy&text={function_name_correct}'
#link = 'https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_period=30&text=Data+science&page=16'
while link:
    print(link)
    html = get_html(link, header)
    #html = get_html_proxies(link, header)
    result = result + parse_hh(html)
    link = get_next_link_hh(html)
    
link = main_link_sj + f'/vacancy/search/?keywords={function_name}&geo[c][0]=1'
while link:
    print(link)
    html = get_html(link, header)
    #html = get_html_proxies(link, header)
    result = result + parse_sj(html)
    link = ''
    link = get_next_link_sj(html)

https://hh.ru/search/vacancy?search_period=30&salary=&st=searchVacancy&text=python
200
https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_period=30&text=python&page=1
200
https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_period=30&text=python&page=2
200
https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_period=30&text=python&page=3
200
https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_period=30&text=python&page=4
200
https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_period=30&text=python&page=5
200
https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_period=30&text=python&page=6
200
https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_period=30&text=python&page=7
200
https://hh.ru/search/vacancy?L_

In [15]:
len(result)

2075

In [16]:
import json
with open("Lesson2_result.json", "w", encoding="utf-8") as file:
    json.dump(result, file)

In [17]:
import pandas as pd

In [18]:
data = pd.DataFrame(result)

In [19]:
data

,name,href,company,position,site,finance_min,finance_max,valuta
0,Python Developer,https://chelyabinsk.hh.ru/vacancy/33738779?que...,iTechArt Group,"Минск, Институт Культуры",https://hh.ru,no data,no data,no data
1,Data Scientist,https://chelyabinsk.hh.ru/vacancy/35011897?que...,HeadHunter::Analytics/Data Science,"Москва, Алексеевская",https://hh.ru,200 000,-,руб.
2,Junior Python Developer (Mogilev),https://chelyabinsk.hh.ru/vacancy/35581311?que...,iTechArt Group,Могилев,https://hh.ru,no data,no data,no data
3,Программист Python/Django,https://chelyabinsk.hh.ru/vacancy/35554799?que...,ТОО BaiBak Exust,Алматы,https://hh.ru,no data,no data,no data
4,Python-разработчик в Процессы бэк-офиса,https://chelyabinsk.hh.ru/vacancy/34950926?que...,Яндекс,Москва,https://hh.ru,no data,no data,no data
5,Middle Python Developer (Mogilev),https://chelyabinsk.hh.ru/vacancy/33576421?que...,iTechArt Group,Могилев,https://hh.ru,no data,no data,no data
6,Software Test Engineer (Python),https://chelyabinsk.hh.ru/vacancy/35586315?que...,ООО СК хайникс мемори солюшнс Восточная Европа,"Минск, Немига",https://hh.ru,no data,no data,no data
7,Test Automation Engineer (Python),https://chelyabinsk.hh.ru/vacancy/35164556?que...,ЗАО Научсофт,Минск,https://hh.ru,no data,no data,no data
8,Аналитик (команда Поиска),https://chelyabinsk.hh.ru/vacancy/35121709?que...,HeadHunter::Analytics/Data Science,"Москва, Алексеевская",https://hh.ru,no data,no data,no data
9,Специалист по анализу данных,https://chelyabinsk.hh.ru/vacancy/35613327?que...,Леруа Мерлен. Центральный офис,Москва,https://hh.ru,no data,no data,no data
